In [22]:
import tensorflow as tf
import os
import glob
import time

print(f"Đang sử dụng TensorFlow phiên bản: {tf.__version__}")

Đang sử dụng TensorFlow phiên bản: 2.13.0


In [28]:
import tensorflow as tf
import os
import glob
import time
import zipfile
import json
import io

# =============================================================================
# Ô CODE TỔNG HỢP (V7 - SỬ DỤNG CONCRETE FUNCTION)
# =============================================================================

# --- PHẦN 1: ĐỊNH NGHĨA CLASS (Giữ nguyên) ---
class FinalModelCNN(tf.keras.Model):
    def __init__(self, input_shape_config, num_classes_config, **kwargs):
        super(FinalModelCNN, self).__init__(**kwargs)
        self.input_shape_config = input_shape_config
        self.num_classes_config = num_classes_config
        self.base_model = tf.keras.applications.EfficientNetV2B2(
            weights="imagenet", 
            include_top=False, 
            input_shape=self.input_shape_config
        )
        self.gap = tf.keras.layers.GlobalAveragePooling2D(name="global_avg_pool")
        self.dense1 = tf.keras.layers.Dense(512, use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-5), name="dense_layer_1")
        self.bn1 = tf.keras.layers.BatchNormalization(name="batch_norm_1")
        self.act1 = tf.keras.layers.Activation('relu', name="activation_1")
        self.dropout1 = tf.keras.layers.Dropout(0.3, name="dropout_layer_1")
        self.dense2 = tf.keras.layers.Dense(256, use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(1e-5), name="dense_layer_2")
        self.bn2 = tf.keras.layers.BatchNormalization(name="batch_norm_2")
        self.act2 = tf.keras.layers.Activation('relu', name="activation_2")
        self.dropout2 = tf.keras.layers.Dropout(0.2, name="dropout_layer_2")
        self.dense_output = tf.keras.layers.Dense(self.num_classes_config, activation='linear', dtype='float32', name="output_layer")

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 256, 126, 3], dtype=tf.float32)])
    def call(self, inputs, training=False):
        # Đặt training=False vì chúng ta đang ở chế độ inference
        x = self.base_model(inputs, training=training)
        x = self.gap(x, training=training) 
        x = self.dense1(x)
        x = self.bn1(x, training=training); x = self.act1(x); x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.bn2(x, training=training); x = self.act2(x); x = self.dropout2(x, training=training)
        outputs = self.dense_output(x)
        return outputs

print("Đã định nghĩa xong class FinalModelCNN (với weights='imagenet' và @tf.function).")
print("Bắt đầu chuyển đổi...")

# --- PHẦN 2: CẤU HÌNH VÀ CHUYỂN ĐỔI ---
BASE_DIR = os.getcwd() 
SOURCE_MODEL_DIR = os.path.join(BASE_DIR, "models") 
DEST_TFLITE_DIR = os.path.join(BASE_DIR, "models_tflite") 
MODEL_TEST_NAME = "CNN_from_NPY_no_val.keras"
INPUT_SHAPE = (256, 126, 3) 
NUM_CLASSES = 4
# ------------------

model_path = os.path.join(SOURCE_MODEL_DIR, MODEL_TEST_NAME)
tflite_name = MODEL_TEST_NAME.replace(".keras", ".tflite")
dest_path = os.path.join(DEST_TFLITE_DIR, tflite_name)

dest_path = os.path.normpath(dest_path) 
os.makedirs(os.path.normpath(DEST_TFLITE_DIR), exist_ok=True)

print(f"Model gốc: {model_path}")
print(f"Model TFLite sẽ lưu tại: {dest_path}")

if not os.path.exists(model_path):
    print(f"\n!!! LỖI KIỂM TRA: Không tìm thấy file tại đường dẫn trên. !!!")
else:
    print("\nKiểm tra: Đã tìm thấy file model. Bắt đầu chuyển đổi...")
    try:
        # === BƯỚC 1: TẠO KIẾN TRÚC ===
        print(f"\n[1/4] Đang tạo kiến trúc 'FinalModelCNN' (weights='imagenet')...")
        model = FinalModelCNN(
            input_shape_config=INPUT_SHAPE, 
            num_classes_config=NUM_CLASSES
        )
        print("[1/4] Tạo kiến trúc thành công.")

        # === BƯỚC 2: BUILD MODEL ===
        print("[2/4] Đang build model (model.build)...")
        # Build với batch_size động (None)
        model.build(input_shape=(None, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]))
        print("[2/4] Build model thành công.")

        # === BƯỚC 3: TẢI TRỌNG SỐ ===
        print(f"[3/4] Đang tải trọng số (weights) từ: {model_path}")
        model.load_weights(model_path, by_name=True, skip_mismatch=True) 
        print("[3/4] Tải trọng số thành công (Các 'Warning' bỏ qua lớp là bình thường).")
        
        # === BƯỚC 4: CHUYỂN ĐỔI TFLITE ===
        print("[4/4] Đang khởi tạo bộ chuyển đổi TFLite...")
        
        # === SỬA LỖI: Lấy 'concrete function' ===
        # Lấy hàm 'call' đã được trace bằng @tf.function
        concrete_func = model.call.get_concrete_function()
        
        # Yêu cầu converter dùng hàm này
        converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
        # ======================================
        
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        print("    -> Đang tiến hành lượng tử hóa (convert)...")
        tflite_model = converter.convert()
        print("[4/4] Chuyển đổi thành công.")
        
        print(f"    -> Đang lưu mô hình TFLite tại: {dest_path}")
        with open(dest_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"\n*** THÀNH CÔNG! Đã tạo file: {tflite_name} (Kích thước: {os.path.getsize(dest_path)} bytes) ***")
    
    except Exception as e:
        print(f"\n!!! LỖI TOÀN BỘ KHI CHUYỂN ĐỔI: {e} !!!")
        print("--- Gợi ý: Lỗi xảy ra trong quá trình tạo kiến trúc hoặc load_weights. ---")

Đã định nghĩa xong class FinalModelCNN (với weights='imagenet' và @tf.function).
Bắt đầu chuyển đổi...
Model gốc: e:\NCKH\dungcocach\Web_NGT\models\CNN_from_NPY_no_val.keras
Model TFLite sẽ lưu tại: e:\NCKH\dungcocach\Web_NGT\models_tflite\CNN_from_NPY_no_val.tflite

Kiểm tra: Đã tìm thấy file model. Bắt đầu chuyển đổi...

[1/4] Đang tạo kiến trúc 'FinalModelCNN' (weights='imagenet')...
[1/4] Tạo kiến trúc thành công.
[2/4] Đang build model (model.build)...
[2/4] Build model thành công.
[3/4] Đang tải trọng số (weights) từ: e:\NCKH\dungcocach\Web_NGT\models\CNN_from_NPY_no_val.keras


c:\Python38\lib\site-packages\keras\src\saving\saving_lib.py:513: UserWarning: Could not load weights in object <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000021542627EE0>. Skipping object. Exception encountered: Layer 'stem_conv' expected 1 variables, but received 0 variables during loading. Expected: ['stem_conv/kernel:0']
  _load_state(
c:\Python38\lib\site-packages\keras\src\saving\saving_lib.py:513: UserWarning: Could not load weights in object <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x000002153DDBE040>. Skipping object. Exception encountered: Layer 'stem_bn' expected 4 variables, but received 0 variables during loading. Expected: ['stem_bn/gamma:0', 'stem_bn/beta:0', 'stem_bn/moving_mean:0', 'stem_bn/moving_variance:0']
  _load_state(
c:\Python38\lib\site-packages\keras\src\saving\saving_lib.py:513: UserWarning: Could not load weights in object <keras.src.layers.convolutional.conv2d.Conv2D object at 0x0000021542620040>. Ski

[3/4] Tải trọng số thành công (Các 'Warning' bỏ qua lớp là bình thường).
[4/4] Đang khởi tạo bộ chuyển đổi TFLite...
    -> Đang tiến hành lượng tử hóa (convert)...
[4/4] Chuyển đổi thành công.
    -> Đang lưu mô hình TFLite tại: e:\NCKH\dungcocach\Web_NGT\models_tflite\CNN_from_NPY_no_val.tflite

*** THÀNH CÔNG! Đã tạo file: CNN_from_NPY_no_val.tflite (Kích thước: 10522008 bytes) ***
